## Wrapping du modèle TensorFlow

Pour des question de compatibilité entre notre modèle TensorFlow et le voting classifier de Scikit learn. Il faut créer wrapper scikit-learn pour le modèle TensorFlow.

In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np

class TensorFlowWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model, preprocess_fn):
        self.model = model
        self.preprocess_fn = preprocess_fn

    def fit(self, X, y=None):
        # On suppose que le modèle est déjà entraîné
        return self

    def predict(self, X):
        X_proc = self.preprocess_fn(X)
        preds = self.model.predict(X_proc)
        return np.argmax(preds, axis=1)

    def predict_proba(self, X):
        X_proc = self.preprocess_fn(X)
        return self.model.predict(X_proc)

def preprocess_EfficientNetB0(image):
    image_pre = tf.keras.applications.efficientnet.preprocess_input(image)
    return image_pre
    

## Wrapper pour gérer le dataset bi-modal

In [12]:
# Séparateurs pour VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.utils.validation import check_is_fitted
import numpy as np

class TextOnly(BaseEstimator, ClassifierMixin):
    def __init__(self, clf=None):
        self.clf = clf

    def fit(self, X, y):
        X_text = [x[0] for x in X]
        self.clf.fit(X_text, y)
        return self

    def predict(self, X):
        X_text = [x[0] for x in X]
        return self.clf.predict(X_text)

    def predict_proba(self, X):
        X_text = [x[0] for x in X]
        return self.clf.predict_proba(X_text)

    def get_params(self, deep=True):
        # Permet à GridSearchCV etc. d'accéder aux params
        return {"clf": self.clf}

    def set_params(self, **params):
        # Permet de setter les params comme clf
        if "clf" in params:
            self.clf = params["clf"]
        return self




class ImageOnly(BaseEstimator, ClassifierMixin):
    def __init__(self, clf):
        self.clf = clf

    def fit(self, X, y):
        X_img = np.stack([x[1] for x in X])
        return self.clf.fit(X_img, y)

    def predict(self, X):
        X_img = np.stack([x[1] for x in X])
        return self.clf.predict(X_img)

    def predict_proba(self, X):
        X_img = np.stack([x[1] for x in X])
        return self.clf.predict_proba(X_img)


## Chargement des modèles pré-entrainés

In [4]:
import joblib
import tensorflow as tf

# Chargement du modèle pour le texte
svm_pipeline = joblib.load('inference_pipeline_svc_model.joblib')
model_img = tf.keras.models.load_model('../../models/EfficientNetB0/EfficientNetB0_model_finetuned_best.keras')


I0000 00:00:1748723990.883758    1122 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:01:00.0, compute capability: 8.9
/home/karim/SEP24_CDS_Rakuten/env/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 24 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


## Création du voting classifier

In [13]:
from sklearn.ensemble import VotingClassifier

wrapped_text_model = TextOnly(clf=svm_pipeline)
wrapped_image_model = ImageOnly(TensorFlowWrapper(model_img, preprocess_EfficientNetB0))

voting = VotingClassifier(
    estimators=[
        ('text_model', wrapped_text_model),
        ('image_model', wrapped_image_model) 
    ],
    voting='soft'   # ou 'hard'
)



## Génération du dataset de test

In [9]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
import pandas as pd
import os

def process_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    x = image.img_to_array(img)
    #x = np.expand_dims(x, axis=0) # Ajout d'une dimension de batch
    x = preprocess_input(x)
    return x
    
df = pd.read_csv('../../data/processed/clean_dataset.csv')

# Ou prendre un pourcentage (ex: 10% du DataFrame)
df_echantillon = df.sample(n=1000) #frac=0.1)

dir_name = "/mnt/c/Users/karim/rakuten/images/data_clean/image_train"
X = []
y = []


for idx, row in df_echantillon.iterrows():
    text_input = row["merged"]
    image_input = process_image(os.path.join(dir_name, f"image_{row["imageid"]}_product_{row["productid"]}.jpg"))
    label = row["prdtypecode"] # label

    X.append((text_input, image_input))
    y.append(label)

# Encoder la cible 
label_encoder = joblib.load('label_encoder.joblib')
y = label_encoder.transform(y)

## Cross validation

In [15]:
from sklearn.metrics import classification_report

y_pred_text = wrapped_text_model.predict(X)
y_pred_image = wrapped_image_model.predict(X)
y_pred_voting = voting.predict(X)

# Évaluation
print(classification_report(y, y_pred_text, target_names=label_encoder.classes_.astype(str)))
print(classification_report(y, y_pred_image, target_names=label_encoder.classes_.astype(str)))
print(classification_report(y, y_pred_voting, target_names=label_encoder.classes_.astype(str)))

2025-05-31 22:45:15.242003: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 602112000 exceeds 10% of free system memory.


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step


NotFittedError: This VotingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [11]:
from sklearn.model_selection import cross_val_score

scores_sklearn = cross_val_score(wrapped_text_model, X, y, cv=5, scoring='accuracy')
print("Sklearn avg acc:", scores_sklearn.mean())

#scores_tf = cross_val_score(wrapped_image_model, X, y, cv=5, scoring='accuracy')
#print("TF avg acc:", scores_tf.mean())

scores_voting = cross_val_score(voting, X, y, cv=5, scoring='accuracy', error_score='raise')
print("Voting avg acc:", scores_voting.mean())





Sklearn avg acc: 0.554


ValueError: The estimator TextOnly should be a classifier.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

cv = KFold(n_splits=5, random_state=111, shuffle=True)

classifiers = [wrapped_text_model, wrapped_image_model, voting]
classifiers_names = ['SVM', 'EfficientNetB0', 'Voting Classifier']

for clf, label in zip(classifiers, classifiers_names):
    scores = cross_validate(clf, X, y, cv=cv, scoring=['accuracy','f1'])
    print("[%s]: \n Accuracy: %0.2f (+/- %0.2f)" % (label, scores['test_accuracy'].mean(), scores['test_accuracy'].std()),
          "F1 score: %0.2f (+/- %0.2f)" % (scores['test_f1'].mean(), scores['test_f1'].std()))